In [ ]:
# !pip install tensorflow==2.3.0
# !pip install tensorflow==2.12.0
!pip install keras==2.10.0 tensorflow==2.10.0
!pip install gym
# !pip install keras
!pip install keras-rl2

In [ ]:
# setup a random prebuilt environment

# import dependencies
import gym, random

In [ ]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [ ]:
# visualize what is actually happening
# aka build the environment

# env.step() returns 5 values instead of 4
# terminated=True if environment terminates (eg. due to task completion, failure etc.)
# truncated=True if episode truncates due to a time limit or a reason that is not defined as part of the task MDP.

episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done_terminated = False
    score = 0

    while not done_terminated:
        env.render()
        action = random.choice([0, 1])
        n_state, reward, done_terminated, info = env.step(action)
        score += reward
    print(f'episode: {episode}, score: {score}')

In [ ]:
# import dependencies
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

In [ ]:
# build the model that will learn to take the best action based on maximizing the reward
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
model = build_model(states, actions)
model.summary()

In [ ]:
# import dependencies
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
import tensorflow as tf

dqn = build_agent(model, actions)
dqn.compile(optimizer=Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

In [ ]:
# just to visualize
_ = dqn.test(env, nb_episodes=15, visualize=True)

In [ ]:
# save model
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
del model
del dqn
del env

In [ ]:
# rebuild environment
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]

# rebuild model
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(optimizer=Adam(learning_rate=1e-3), metrics=['mae'])

# reload weights
dqn.load_weights('dqn_weights.h5f')

# test out environment
_ = dqn.test(env, nb_episodes=5, visualize=True)